In [1]:
import pandas as pd
import sqlalchemy as sa
from openpyxl.cell import Cell

In [2]:
# read db connection from .env file
from helpers.settings import get_settings

settings = get_settings()

Loading settings ...


In [3]:
engine = sa.create_engine(settings.postgresql_dsn.__str__())

In [9]:
# find the current working directory
import os
import IPython

notebook_path = IPython.get_ipython().getoutput('pwd')[0]

data_dir = os.path.join(notebook_path, '..', 'data', "analysis_3")
fig_dir = os.path.join(notebook_path, '..', 'figures', "analysis_3")

if not os.path.exists(data_dir):
    print(f'Creating directory: {data_dir}')
    os.makedirs(data_dir)

if not os.path.exists(fig_dir):
    print(f'Creating directory: {fig_dir}')
    os.makedirs(fig_dir)

Creating directory: /Users/ac4294/dev/2024/cpvt_database_analysis/notebooks/../figures/analysis_3


In [6]:
seq_variants = pd.read_sql("""
SELECT sv.sequence_variant_id, sv.c_hgvs_string, sv.p_hgvs_string
FROM sequence_variant sv
WHERE sequence_variant_id IN (
    SELECT sv2.sequence_variant_id
    FROM sequence_variant sv2
    JOIN variant v2
    ON sv2.sequence_variant_id = v2.sequence_variant_id
    JOIN individual_variant iv2
    ON v2.variant_id = iv2.variant_id
)

""", engine)
# the sequence variants associated with the individuals
seq_variants.head()

,sequence_variant_id,c_hgvs_string,p_hgvs_string
0,1,NM_001035.3:c.6737C>T,NP_001026.2:p.(Ser2246Leu)
1,2,NM_001035.3:c.7422G>C,NP_001026.2:p.(Arg2474Ser)
2,4,NM_001035.3:c.13489C>T,NP_001026.2:p.(Arg4497Cys)
3,5,NM_001035.3:c.7157A>T,NP_001026.2:p.(Asn2386Ile)
4,6,NM_001035.3:c.1298T>C,NP_001026.2:p.(Leu433Pro)


In [7]:
seq_variants.count()

sequence_variant_id    392
c_hgvs_string          341
p_hgvs_string          382
dtype: int64

In [12]:
# export to csv
seq_variants["variant"] = seq_variants.apply(
    lambda x: x["c_hgvs_string"] if x["c_hgvs_string"] is not None else x["p_hgvs_string"],
    axis=1
)
seq_variants.to_csv(os.path.join(data_dir, 'sequence_variants.csv'), index=False)

In [ ]:
# load in the 